In [27]:
import pandas as pd
import numpy as np
import datetime
import quandl
import matplotlib.pyplot as plt

In [28]:
tcs_df = pd.read_csv('final_tcs.csv')

In [29]:
tcs_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,DOWJONES,NASDAQ,NSE
0,2017-03-14,2556.800049,2587.149902,2551.050049,2567.500000,2519.130615,1701147,20837.369141,5856.819824,9087.000000
1,2017-03-15,2572.000000,2572.149902,2493.850098,2498.300049,2451.234131,1511248,20950.099609,5900.049805,9084.799805
2,2017-03-16,2506.000000,2527.000000,2506.000000,2520.899902,2473.408447,785909,20934.550781,5900.759766,9153.700195
3,2017-03-17,2546.000000,2546.000000,2509.750000,2525.750000,2478.167236,1296576,20914.619141,5901.000000,9160.049805
4,2017-03-20,2525.949951,2525.949951,2466.000000,2480.000000,2433.279297,1743770,20905.859375,5901.529785,9126.849609


In [30]:
#def allStockRsi(allStocks,data,ndays):
def stockRsi(data):
    delta = data['Close'].diff()
    deltaUp = np.where(delta>0, delta, 0)
    deltaDown = np.where(delta<0,delta,0)
    dfDeltaUp = pd.DataFrame(data=deltaUp,columns=['deltaUp'])
    rollingDf = pd.DataFrame(np.zeros,index=range(0,len(data['Close'])),columns=['rollingUp','rollingDown'])
    rollingDf['rollingUp'] = dfDeltaUp.rolling(window=20).mean()
    rollingDf['rollingUp'].fillna(0,inplace=True)
    dfDeltaDown = pd.DataFrame(data=deltaDown,columns=['deltaDown'])
    rollingDf['rollingDown'] = dfDeltaDown.rolling(window=20).mean()
    rollingDf['rollingDown'].fillna(0,inplace=True)
    rollingDf['RS'] = (rollingDf['rollingUp']/rollingDf['rollingDown']).abs()
    rsi = 100.0 - (100.0/(1.0 + rollingDf['RS']))
        #df_temp.REMARK.apply(lambda v: 1 if str(v)!='nan' else 0)
        #data.ix[stock]['RSI'] = rsi.apply(lambda v: v if str(v)!='nan' else 0)
    data['RSI'] = rsi
    return data


In [47]:
#Function for difference between current day's opening price and previous day's closing price
def pcod(stock):
    stock['pcod'] = stock['Open'] - stock['Close'].shift(1)
    return stock

In [54]:
#Function for difference between current day's highest price and lowest price
def hld(stock):
    stock['hld'] = stock['High'] - stock['Low']
    return stock

In [79]:
#Function to calculate bollinger bands average, upper and lower bands
def bbands(stock, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    ave = stock['Close'].rolling(window=length).mean()
    sd = stock['Close'].rolling(window=length).std()
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    stock['ave'], stock['upperBD'], stock['lowerBD'] = np.round(ave,3), np.round(upband,3), np.round(dnband,3)
    return stock

In [64]:
#Function for simple day moving average
def sma(stock,numOfDays):
    stock['sma'] = stock['Close'].rolling(window=numOfDays).mean()
    return stock

In [68]:
#Function for exponential moving average 
def ema(stock,numOfDays):
    stock['ema'] = stock['Close'].ewm(span=numOfDays,adjust=False,min_periods=numOfDays-1).mean()
    return stock

In [83]:
#Apply all functions one by one
tcs_df = stockRsi(tcs_df)
tcs_df = pcod(tcs_df)
tcs_df = hld(tcs_df)
tcs_df = bbands(tcs_df, length=20, numsd=1)
tcs_df = sma(tcs_df,20)
tcs_df = ema(tcs_df,20)

In [84]:
tcs_df = ema(tcs_df,20)

In [85]:
tcs_df.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,DOWJONES,NASDAQ,NSE,RSI,pcod,hld,ave,upperBD,lowerBD,sma,ema
0,2017-03-14,2556.800049,2587.149902,2551.050049,2567.500000,2519.130615,1701147,20837.369141,5856.819824,9087.000000,NaN,NaN,36.099853,NaN,NaN,NaN,NaN,NaN
1,2017-03-15,2572.000000,2572.149902,2493.850098,2498.300049,2451.234131,1511248,20950.099609,5900.049805,9084.799805,NaN,4.500000,78.299804,NaN,NaN,NaN,NaN,NaN
2,2017-03-16,2506.000000,2527.000000,2506.000000,2520.899902,2473.408447,785909,20934.550781,5900.759766,9153.700195,NaN,7.699951,21.000000,NaN,NaN,NaN,NaN,NaN
3,2017-03-17,2546.000000,2546.000000,2509.750000,2525.750000,2478.167236,1296576,20914.619141,5901.000000,9160.049805,NaN,25.100098,36.250000,NaN,NaN,NaN,NaN,NaN
4,2017-03-20,2525.949951,2525.949951,2466.000000,2480.000000,2433.279297,1743770,20905.859375,5901.529785,9126.849609,NaN,0.199951,59.949951,NaN,NaN,NaN,NaN,NaN
5,2017-03-21,2482.000000,2492.000000,2465.000000,2486.699951,2439.852783,1686850,20668.009766,5793.830078,9121.500000,NaN,2.000000,27.000000,NaN,NaN,NaN,NaN,NaN
6,2017-03-22,2480.149902,2496.000000,2469.949951,2478.949951,2432.249023,944358,20661.300781,5821.640137,9030.450195,NaN,-6.550049,26.050049,NaN,NaN,NaN,NaN,NaN
7,2017-03-23,2482.000000,2485.050049,2447.199951,2460.350098,2413.999268,1310319,20656.580078,5817.689941,9086.299805,NaN,3.050049,37.850098,NaN,NaN,NaN,NaN,NaN
8,2017-03-24,2469.800049,2469.800049,2423.300049,2427.199951,2381.473877,2549907,20596.720703,5828.740234,9108.000000,NaN,9.449951,46.500000,NaN,NaN,NaN,NaN,NaN
9,2017-03-27,2426.949951,2440.000000,2406.000000,2412.050049,2366.609131,1090635,20550.980469,5840.370117,9045.200195,NaN,-0.250000,34.000000,NaN,NaN,NaN,NaN,NaN
